# Lesson 1: Simple ReAct Agent from Scratch

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [3]:
# !pip install openai python-dotenv

### Basically, we are enabling the agent to iteratively perform actions and make decisions based on environmental feedback (pre, self, defined).

## reasoning + acting

Action is executed in an environment, and an observation returns. With that action, the LLM repeats, thinks about what to do again, and then decides another action to take, until it decides it's time to be done.

In [5]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [6]:
oepnai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
client = OpenAI(api_key = oepnai_api_key)
# different syntex again
# initialize the llm

In [9]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hi, my name is Alice. What llm are you based on? Introduce yourself in 50 words."}]
)

In [10]:
chat_completion

ChatCompletion(id='chatcmpl-AF5QLIgUaD83mz0BULl5iVlGByIrr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello Alice, I am an AI digital assistant based on OpenAI's language model. I am here to assist you with any queries or tasks you may have. Please feel free to ask me anything, and I will do my best to help you.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728159133, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=50, prompt_tokens=30, total_tokens=80, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [11]:
chat_completion.choices[0].message.content

"Hello Alice, I am an AI digital assistant based on OpenAI's language model. I am here to assist you with any queries or tasks you may have. Please feel free to ask me anything, and I will do my best to help you."

## create agent, parameterized by a system message, we will allow the user to pass that in and we will save that as an attribute. We will also keep track of the messages.

### will also add system message.

### call method and execute methods. Take a message as string and then append the message to the existing array of messages. We will then execute a function.

### We will then call the execute, define the completion here and then return the message.

In [12]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content


#### react agent will need a system of message and egents.

#### thought, action, pause and action.

In [17]:
# the prompt basiclaly gives instructions to the agent.
# output an answer, use thought to describe your thoughts and the questions and actions to execute.
# .... just words to articulate the consequences.

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

# Answer: ..... this is the result format

In [18]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [19]:
abot = Agent(prompt)

In [20]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using average_dog_weight.
Action: average_dog_weight: Toy Poodle
PAUSE


In [21]:
result = average_dog_weight("Toy Poodle")

In [22]:
result

'a toy poodles average weight is 7 lbs'

In [23]:
result2 = average_dog_weight("a kitten")

In [24]:
result2

'An average dog weights 50 lbs'

In [25]:
next_prompt = "Observation: {}".format(result)

In [28]:
next_prompt2 = "Observation: {}".format(result2)

In [29]:
abot(next_prompt2)

'Answer: An average dog weighs 50 lbs, but a Toy Poodle specifically weighs an average of 7 lbs.'

In [30]:
abot(next_prompt)

'Answer: A Toy Poodle weighs an average of 7 lbs.'

In [31]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [32]:
abot = Agent(prompt)

In [33]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [34]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [35]:
abot(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [36]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [37]:
abot(next_prompt)

'Action: calculate: 37 + 20\nPAUSE'

In [38]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [39]:
abot(next_prompt)

'Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Add loop

In [40]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [41]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [43]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
I also have a kitten.\
I also have a tank of fish. \
What is their combined weight"""
query(question)

## the knowledge about a tank of fish and a kitten is the common knowlegde retrived from the llm

Thought: To find the combined weight of the pets, I need to find the average weight of a Border Collie and a Scottish Terrier. I will also need to estimate the weight of a kitten and the fish tank. I will start by finding the average weight of the Border Collie and the Scottish Terrier.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have the average weights of the Border Collie and the Scottish Terrier, I need to estimate the weight of a kitten and the fish tank. A typical kitten weighs around 2 to 4 lbs, and a small fish tank with water and fish might weigh around 10 to 20 lbs. I will use an average weight of 3 lbs for the kitten and 15 lbs for the fish tank for this calculation.

Action: calculate: 37 + 20 + 3 + 15
PAUSE


### Put it together

In [128]:
# just functions without the agent part yet
def get_feed_back():
    mood = input("How are you feeling? (happy/sad/...) ")
    return f"I see, you're feeling {mood}."

def extract_mood(feedback):
    # have some options
    moods = ["happy", "sad", "angry", "anxious", "relaxed", "excited"]
    for mood in moods:
        if mood in feedback:
            return mood
    return "neutral"

def routine(time, mood):
    if "happy" or "relaxed" or "excited"in mood:
        if "early morning" in time:
            return "wake up, brush your teeth, and eat breakfast."
        elif "late morning" in time:
            return "get dressed, go to work."
        elif "mid afternoon" in time:
            return "Have lunch."
        elif "late afternoon" in time:
            return "take a break, and have dinner."
        elif "early evening" in time:
            return "Have a snack, go to bed."
        else:
            return "Do whatever you want."
    elif "sad" or "angry" or "anxious" in mood:
        return "XXXXXXXX"
    else:
        return "I don't know what to suggest."

known_actions = {
    "get_feed_back": get_feed_back,
    "routine": routine
}

In [129]:
def interaction_loop():
    state = "initial"
    mood = ""
    time = ""

    while True:
        if state == "initial":
            print("question: How are you feeling?")
            print("Thought: I am collecting the user's feedback about their mood.")
            action = "get_feed_back"
            result = known_actions[action]()
            print(f"observation: {result}")

            mood = extract_mood(result)
            state = "get_time"

        elif state == "get_time":
            print("question: What time of day is it?")
            time = input("enter the time of day: ")
            print(f"Thought: I want to know the time to provide the correct routine for someone feeling {mood}.")

            action = "routine"
            result = known_actions[action](time, mood)
            state = "routine_suggestion"

        elif state == "routine_suggestion":
            print(f"observation: {result}")
            print(f"Answer: {result}")
            break

In [98]:
# Run the interaction loop
interaction_loop()

question: How are you feeling?
Thought: I am collecting the user's feedback about their mood.
How are you feeling? (happy/sad/...) happy
observation: I see, you're feeling happy.
question: What time of day is it?
enter the time of day: early morning
Thought: I want to know the time to provide the correct routine for someone feeling happy.
observation: wake up, brush your teeth, and eat breakfast.
Answer: wake up, brush your teeth, and eat breakfast.


In [95]:
# Run the interaction loop
interaction_loop()

question: How are you feeling?
Thought: I am collecting the user's feedback about their mood.
How are you feeling? (happy/sad/...) sad
observation: I see, you're feeling sad.
question: What time of day is it?
enter the time of day: evening
Thought: I want to know the time to provide the correct routine for someone feeling sad.
observation: Do whatever you want.
Answer: Do whatever you want.


In [100]:
# previous original codes
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

abot = Agent(prompt)

result = abot("How much does a toy poodle weigh?")
print(result)

result = average_dog_weight("Toy Poodle")
next_prompt = "Observation: {}".format(result)
abot(next_prompt)
abot.messages
abot = Agent(prompt)
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)
abot(next_prompt)
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)
abot(next_prompt)
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)
abot(next_prompt)
action_re = re.compile('^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I should look up the average weight of a Toy Poodle using average_dog_weight.
Action: average_dog_weight: Toy Poodle
PAUSE
Observation: a Border Collies average weight is 37 lbs
Observation: Scottish Terriers average 20 lbs
Observation: 57
Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to find their combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.


In [130]:
# new functions
def get_feedback():
    return input("How are you feeling? ")

def routine(time, mood):
    if "happy" in mood:
        if "early morning" in time:
            return "Wake up, brush your teeth, and eat breakfast."
        elif "late morning" in time:
            return "Get dressed, go to work."
        elif "mid afternoon" in time:
            return "Have lunch."
        elif "late afternoon" in time:
            return "Take a break, and have dinner."
        elif "early evening" in time:
            return "Have a snack, go to bed."
        else:
            return "Do whatever you want."
    elif "sad" in mood:
        return "Get some sleep."
    else:
        return "I'm not sure what to suggest."

In [131]:
known_actions = {
    "get_feedback": get_feedback,
    "routine": routine
}

In [132]:
action_re = re.compile(r'^Action: (\w+): (.*)$')

In [138]:
class Agent:
    def __init__(self, prompt):
        self.prompt = prompt
        self.messages = []
        self.final_answer = None

    def __call__(self, input):
        self.messages.append(f"question: {input}")
        thought = self.think(input)
        self.messages.append(f"Thought: {thought}")

        action, action_input = self.get_action(thought)
        if action:
            self.messages.append(f"Action: {action}: {action_input}")
            return "PAUSE"

        return self.final_answer if self.final_answer else "Answer: I don't know what to do."

    def think(self, question):
        if "mood" in question or "feeling" in question:
            return "I should ask the user about their mood using get_feedback."
        elif "morning" in question or "afternoon" in question or "evening" in question:
            return "I should suggest a routine using the routine function."
        else:
            return "I don't know what to suggest."

    def get_action(self, thought):
        if "get_feedback" in thought:
            return "get_feedback", ""
        elif "routine" in thought:
            return "routine", "early morning happy"
        return None, None

    def set_final_answer(self, answer):
        self.final_answer = f"Answer: {answer}"

In [143]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question

    while i < max_turns:
        i += 1
        result = bot(next_prompt)

        if result == "PAUSE":
            actions = [
                action_re.match(a)
                for a in bot.messages[-1].split('\n')
                if action_re.match(a)
            ]

            if actions:
                action, action_input = actions[0].groups()
                if action not in known_actions:
                    raise Exception(f"unknown action: {action}: {action_input}")
                print(f" now running {action} {action_input}")
                if action == "get_feedback":
                    observation = known_actions[action]()
                else:
                    time = input("What time of day is it? (early morning, late morning, mid afternoon, etc.): ")
                    mood = known_actions["get_feedback"]()
                    observation = known_actions[action](time, mood)

                print(f"Observation: {observation}")

                bot.set_final_answer(observation)

                next_prompt = f"Observation: {observation}"
        else:
            print(result)
            return

In [144]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_feedback:
e.g. get_feedback
Gets the user's feedback.

routine:
e.g. routine: early morning happy
Returns a suggested routine based on time of day and mood.
"""

In [145]:
question = "What should I do this morning?"
query(question)

 now running routine early morning happy
What time of day is it? (early morning, late morning, mid afternoon, etc.): early morning
How are you feeling? happy
Observation: Wake up, brush your teeth, and eat breakfast.
Answer: Wake up, brush your teeth, and eat breakfast.
